In [16]:
DATADIR="/home/s1620444/data1/data"
DATASETSDIR="/home/s1620444/data1/data/tfds"
RESULTSDIR="/home/s1620444/data1/results"
WEIGHTSDIR="/home/s1620444/data1/results/weights"
PROJECTDIR="/data1/s1620444/automl-sr-rs/autoSR"

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
from tqdm import tqdm,trange

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

import tensorflow_datasets as tfds
import sys
import numpy as np

os.environ["DATASETSDIR"]=DATASETSDIR
os.environ["RESULTSDIR"]=RESULTSDIR
os.environ["DATADIR"]=DATADIR

sys.path.append(PROJECTDIR)
from utils import get_data,resolve
from baselines.wdsr.wdsr import wdsr_b
from utils import psnr,ssim
from baselines.rcan.rcan import generator as rcan
from auto_models.models.autosrcnn import autoSRCNN
from auto_models.models.autosr_rs import autoSR_RS


In [17]:
# load wdsr & rcan
def load_model(model_name: str, train_dataset: str, id: int, weights_dir: str):
    if train_dataset == "oli2msi":
        SCALE=3
    else:
        SCALE=2

    dataset_options=["cerrado","sr_ucmerced","sr_so2sat","oli2msi"]

    if model_name=="wdsr":
        model=wdsr_b(num_res_blocks=32, scale=SCALE)
        model.load_weights(f"{weights_dir}/wdsr-b-{train_dataset}-32-x{SCALE}-{id}/final_model.h5")
    elif model_name=="rcan":
        model=rcan(scale=SCALE)
        # extract weights from nested checkpoint structure
        model_dir=f"{weights_dir}/rcan-{train_dataset}-x{SCALE}-{id}"
        checkpoint_subdirs=os.listdir(model_dir)
        checkpoint_subdirs.sort()
        # sometimes there are no checkpoints, just the final model
        if checkpoint_subdirs[-1] == "final_model.h5":
            subdir=""
        else:
            subdir=checkpoint_subdirs[-1]
        model.load_weights(os.path.join(model_dir,subdir, "final_model.h5"))
    elif model_name=="autosrcnn":
        model=autoSRCNN(os.path.join(RESULTSDIR,"autosrcnn"), f"{id}_simple_srcnn_{train_dataset}", max_trials=20, scale=SCALE, overwrite=False)
    elif model_name=="autosr":
        if dataset in dataset_options:
            dataset_options.remove(dataset)
            # autoSR_RS(dataset_options,trained_weights_dir,f"0_autosr_v1_{dataset}", "v1",max_trials=20, scale=scale, overwrite=False)
        model=autoSR_RS(dataset_options,os.path.join(RESULTSDIR,"autosr/v1"), f"{id}_autosr_v1_{train_dataset}","v1", max_trials=20, scale=SCALE, overwrite=False)
    return model


In [18]:
def evaluate_model(model, model_name, dataset):
    if model_name =="autosrcnn" or model_name=="autosr":
        psnr_values = []
        ssim_values=[]
        
        for lr,hr in dataset.take(100):
            sr = tf.convert_to_tensor(np.around(np.clip(model.predict(lr.numpy(), custom_objects={"psnr":psnr,"ssim":ssim}),0,255)).astype("uint8")[0],np.uint8)
            psnr_values.append(tf.keras.backend.get_value(psnr(sr,hr))[0])
            ssim_values.append(tf.keras.backend.get_value(ssim(sr,hr))[0])

        return np.array(psnr_values), np.array(ssim_values)
        
    # rcan & wdsr
    psnr_values=[]
    ssim_values=[]
    for lr, hr in dataset.take(100):
        sr= resolve(model, lr)
        psnr_values.append(tf.keras.backend.get_value(psnr(hr,sr))[0])
        ssim_values.append(tf.keras.backend.get_value(ssim(hr,sr))[0])

    return np.array(psnr_values), np.array(ssim_values)

In [19]:
def get_test_set(dataset_name,batch_size, as_supervised, as_numpy):
    test,_,_=get_data(dataset_name,train_batch_size=-1, test_batch_size=batch_size, val_batch_size=-1, as_supervised=as_supervised)
    if as_numpy:
        return tfds.as_numpy(test)
    else:
        test.repeat(1)
        return test

In [21]:
datasets=["cerrado","sr_ucmerced","oli2msi","sent_nicfi"]
models=["wdsr","rcan","autosr","autosrcnn"]

datasets_bar=tqdm(datasets)
datasets_bar.set_description("train_data")
models_bar=tqdm(models)
models_bar.set_description("model")


# f.write("model,dataset,psnr,ssim\n")
for model_name in models_bar:
    batch_size=1
    as_supervised=True
    as_numpy=False
    for dataset in datasets_bar:
        test = get_test_set(dataset,batch_size, as_supervised,as_numpy)
        model=load_model(model_name, dataset, 0, WEIGHTSDIR)
        psnr_vals,ssim_vals= evaluate_model(model, model_name, test)
        print(model_name,dataset, psnr_vals, ssim_vals)
        with open(f"{RESULTSDIR}/test_scores_per_img_sample.csv", "a") as f:
            f.write(",".join([model_name, dataset, "\"["+",".join(str(i) for i in psnr_vals)+"]\"","\"["+",".join(str(i) for i in ssim_vals)+"]\""])+"\n")


train_data:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Reloading Oracle from existing project /home/s1620444/data1/results/autosrcnn/0_simple_srcnn_cerrado_trial_20/oracle.json


INFO:tensorflow:Reloading Oracle from existing project /home/s1620444/data1/results/autosrcnn/0_simple_srcnn_cerrado_trial_20/oracle.json


INFO:tensorflow:Reloading Tuner from /home/s1620444/data1/results/autosrcnn/0_simple_srcnn_cerrado_trial_20/tuner0.json


INFO:tensorflow:Reloading Tuner from /home/s1620444/data1/results/autosrcnn/0_simple_srcnn_cerrado_trial_20/tuner0.json


1/1 [==============================] - 0s 95ms/step


model: 100%|██████████| 1/1 [03:03<00:00, 183.21s/it]

autosrcnn cerrado [40.440903 33.124386 37.195965 42.79157  35.770744 35.639477 35.705532
 38.37733  41.576115 42.612263 37.911007 40.287876 35.15385  39.174854
 35.31135  38.097775 34.616585 39.883137 38.662518 36.18476  40.640396
 37.04793  38.19916  39.9777   35.075462 40.603268 38.642117 32.121925
 40.739914 39.31578  33.409676 41.38348  38.70991  36.82414  40.261356
 38.127937 37.896282 35.666683 35.111324 38.232967 36.67672  42.914223
 38.548504 40.54169  39.27686  35.71941  36.873653 34.910007 39.31448
 36.099728 38.954494 38.696205 38.41705  38.307602 38.59203  35.491512
 38.631237       inf 45.802593 41.390137 36.988487 37.15613  37.806572
 39.632717 35.311687 33.550896 37.041847 36.695248 40.903633 36.111683
 40.68185  37.47894  40.141853 39.77136  37.670155 37.10064  40.557915
 39.11486  35.030327 39.716072 34.254417 38.71355  35.43776  40.574883
 39.32047  39.03361  40.24856  40.1204   39.214134 38.586334 38.69556
 35.23849  35.87244  38.42676  35.556377 37.824417 36.921143 